# 深度Q学习(DQN)完整教程

本教程将带你从零开始理解和实现Deep Q-Network算法。

## 目录
1. [理论基础](#1-理论基础)
2. [核心组件](#2-核心组件)
3. [实现DQN Agent](#3-实现dqn-agent)
4. [训练与评估](#4-训练与评估)
5. [算法变体对比](#5-算法变体对比)

## 1. 理论基础

### 1.1 Q-Learning回顾

Q-Learning是一种无模型的强化学习算法，通过学习动作价值函数Q(s,a)来找到最优策略。

**贝尔曼最优方程**:
$$Q^*(s, a) = \mathbb{E}\left[ r + \gamma \max_{a'} Q^*(s', a') \mid s, a \right]$$

其中:
- $Q^*(s, a)$: 最优动作价值函数
- $r$: 即时奖励
- $\gamma \in [0, 1]$: 折扣因子
- $s'$: 下一状态

### 1.2 为什么需要Deep Q-Network?

传统Q-Learning使用表格存储Q值，面临以下挑战:

1. **维度灾难**: 状态空间指数增长 $|S| = O(d^n)$
2. **缺乏泛化**: 每个状态独立学习，无法迁移知识
3. **连续状态**: 无法处理连续状态空间

**DQN解决方案**: 使用神经网络作为函数逼近器
$$Q(s, a; \theta) \approx Q^*(s, a)$$

### 1.3 DQN的两大创新

#### 经验回放 (Experience Replay)
- 存储转移 $(s, a, r, s', done)$ 到缓冲区
- 随机采样打破时序相关性
- 提高数据利用效率

#### 目标网络 (Target Network)
- 使用独立的目标网络计算TD目标
- 定期同步参数: $\theta^- \leftarrow \theta$
- 提供稳定的回归目标

In [ ]:
# 环境设置
import sys
sys.path.insert(0, '..')

import numpy as np
import torch
import matplotlib.pyplot as plt

# 检查GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')

## 2. 核心组件

### 2.1 经验回放缓冲区

In [ ]:
from buffers import ReplayBuffer

# 创建缓冲区
buffer = ReplayBuffer(capacity=10000)
print(f'缓冲区容量: {buffer.capacity}')
print(f'当前大小: {len(buffer)}')

In [ ]:
# 模拟存储转移
for i in range(100):
    state = np.random.randn(4).astype(np.float32)
    action = np.random.randint(0, 2)
    reward = np.random.randn()
    next_state = np.random.randn(4).astype(np.float32)
    done = np.random.random() < 0.1
    
    buffer.push(state, action, reward, next_state, done)

print(f'存储后大小: {len(buffer)}')

In [ ]:
# 采样批次
if buffer.is_ready(32):
    states, actions, rewards, next_states, dones = buffer.sample(32)
    print(f'状态形状: {states.shape}')
    print(f'动作形状: {actions.shape}')
    print(f'奖励形状: {rewards.shape}')

### 2.2 Q网络架构

In [ ]:
from networks import DQNNetwork, DuelingDQNNetwork

# 标准DQN网络
q_net = DQNNetwork(state_dim=4, action_dim=2, hidden_dims=[128, 128])
print('标准DQN网络:')
print(q_net)

In [ ]:
# Dueling DQN网络
dueling_net = DuelingDQNNetwork(state_dim=4, action_dim=2, hidden_dims=[128, 128])
print('Dueling DQN网络:')
print(dueling_net)

In [ ]:
# 测试前向传播
test_state = torch.randn(1, 4)
q_values = q_net(test_state)
print(f'Q值输出: {q_values}')
print(f'最优动作: {q_values.argmax().item()}')

## 3. 实现DQN Agent

### 3.1 创建Agent

In [ ]:
from agent import DQNAgent, create_dqn_agent
from core import DQNConfig

# 方法1: 使用配置类
config = DQNConfig(
    state_dim=4,
    action_dim=2,
    hidden_dims=[128, 128],
    learning_rate=1e-3,
    gamma=0.99,
    double_dqn=True,
    dueling=False,
)
agent = DQNAgent(config)
print(agent)

In [ ]:
# 方法2: 使用工厂函数
agent = create_dqn_agent(
    state_dim=4,
    action_dim=2,
    double_dqn=True,
    dueling=True,
)
print(agent)

### 3.2 动作选择 (ε-贪婪策略)

In [ ]:
# 测试动作选择
state = np.random.randn(4).astype(np.float32)

# 训练模式 (带探索)
action_train = agent.select_action(state, training=True)
print(f'训练模式动作: {action_train}, epsilon: {agent.epsilon:.3f}')

# 评估模式 (贪婪)
action_eval = agent.select_action(state, training=False)
print(f'评估模式动作: {action_eval}')

### 3.3 训练步骤

In [ ]:
# 模拟训练步骤
losses = []

for i in range(200):
    state = np.random.randn(4).astype(np.float32)
    action = agent.select_action(state, training=True)
    reward = np.random.randn()
    next_state = np.random.randn(4).astype(np.float32)
    done = np.random.random() < 0.1
    
    loss = agent.train_step(state, action, reward, next_state, done)
    if loss is not None:
        losses.append(loss)

print(f'训练步数: {agent.training_step}')
print(f'更新次数: {agent.update_count}')
print(f'当前epsilon: {agent.epsilon:.4f}')

In [ ]:
# 绘制损失曲线
if losses:
    plt.figure(figsize=(10, 4))
    plt.plot(losses)
    plt.xlabel('更新步数')
    plt.ylabel('损失')
    plt.title('训练损失曲线')
    plt.grid(True, alpha=0.3)
    plt.show()

## 4. 训练与评估

### 4.1 在CartPole环境训练

In [ ]:
try:
    import gymnasium as gym
    HAS_GYM = True
except ImportError:
    HAS_GYM = False
    print('gymnasium未安装，跳过环境训练')

In [ ]:
if HAS_GYM:
    from train import train_dqn, evaluate_agent
    from utils.training import TrainingConfig
    
    # 创建agent
    agent = create_dqn_agent(
        state_dim=4,
        action_dim=2,
        double_dqn=True,
        epsilon_decay=5000,
    )
    
    # 训练配置 (快速测试)
    config = TrainingConfig(
        num_episodes=100,  # 生产环境建议300+
        max_steps_per_episode=500,
        log_frequency=20,
        eval_frequency=50,
    )
    
    # 训练
    metrics = train_dqn(agent, 'CartPole-v1', config)

In [ ]:
if HAS_GYM and 'metrics' in dir():
    from utils.visualization import plot_training_curves
    
    plot_training_curves(
        metrics.episode_rewards,
        metrics.losses,
        metrics.epsilon_history,
        metrics.eval_rewards,
        title='DQN训练进度',
    )

## 5. 算法变体对比

### 5.1 Double DQN

**问题**: 标准DQN的max操作导致过估计
$$\mathbb{E}[\max_a Q(s,a)] \geq \max_a \mathbb{E}[Q(s,a)]$$

**解决方案**: 解耦动作选择和评估
$$y = r + \gamma Q(s', \arg\max_{a'} Q(s', a'; \theta); \theta^-)$$

### 5.2 Dueling DQN

**核心思想**: 分解Q函数为状态价值和动作优势
$$Q(s, a) = V(s) + A(s, a) - \frac{1}{|A|} \sum_{a'} A(s, a')$$

**优势**:
- 状态价值从所有动作经验学习
- 在动作选择不重要的状态更快收敛

### 5.3 优先经验回放 (PER)

**核心思想**: 按TD误差大小优先采样
$$P(i) = \frac{p_i^\alpha}{\sum_k p_k^\alpha}, \quad p_i = |\delta_i| + \epsilon$$

**重要性采样权重**:
$$w_i = \left( \frac{1}{N \cdot P(i)} \right)^\beta$$

In [ ]:
from buffers import PrioritizedReplayBuffer

# 创建优先回放缓冲区
per_buffer = PrioritizedReplayBuffer(
    capacity=10000,
    alpha=0.6,  # 优先化程度
    beta_start=0.4,  # 初始重要性采样
)

print(f'PER缓冲区: {per_buffer}')

## 总结

本教程介绍了DQN的核心概念和实现:

1. **经验回放**: 打破时序相关性，提高数据效率
2. **目标网络**: 提供稳定的训练目标
3. **Double DQN**: 解决过估计问题
4. **Dueling DQN**: 价值-优势分解
5. **优先经验回放**: 高效采样重要样本

### 下一步
- 尝试不同的超参数组合
- 在更复杂的环境中测试
- 实现Rainbow DQN (组合所有改进)